In [ ]:
import sys
import os
from os.path import dirname, basename,join, exists
import glob

# operations
import numpy as np
import pandas as pd
import geopandas as gpd
# import flopy

# plotting
# from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt

from osgeo import gdal
import contextily as ctx



In [ ]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
# dir of stream level data for seepage study
proj_dir = gwfm_dir + '/Stream_seepage/'
dat_dir = proj_dir+'Stream_level_data/'

sfr_dir = gwfm_dir+'/SFR_data/'

In [ ]:
flopy_dir = doc_dir+'/GitHub/flopy/'
if flopy_dir not in sys.path:
    sys.path.append(flopy_dir)
# sys.path
import flopy 

from importlib import reload
# importlib.reload
reload(flopy)

In [ ]:
def get_nodes(locs):
    ''' Converts layer, row, column into model node number '''
    nodes = []
    for k, i, j in locs:
        nodes.append(k * m.dis.nrow * m.dis.ncol + i * m.dis.ncol + j)
    return nodes


In [ ]:
def epd_2_df(epd, rot = 0, crs=None):
    """ Provide an Endpoint recarray and rotation of grid (degrees) if one exists"""
    epd_df = pd.DataFrame().from_records(epd)
    # status meaning
    status_dict = {0:'Pending',1:'Active',2:'NormallyTerminated',3:'ZoneTerminated',4:'Unreleased',5:'Stranded'}
    epd_df['status_name'] = [status_dict.get(key) for key in epd_df.status]
    # return particle status
    for sn in epd_df.status_name.unique():
        if (epd_df.status_name==sn).sum() >0:
            print(str((epd_df.status_name==sn).sum())+' particles '+ sn)
    # present age in days
    epd_df['age_days'] = epd_df.time - epd_df.time0
    # age in years
    epd_df['age_years'] = epd_df.age_days/365
    if tracking == 'forward':
        loc_id = '' # with backward tracking final location is source
    elif tracking =='backward':
        loc_id = '0' # with forward tracking initial location is source
    # identify source row/column 
    epd_df['j'] = (epd_df['x'+loc_id]/delr-0.5).astype(int)
    epd_df['i'] = (epd_df['y'+loc_id]/delc-0.5).astype(int)
    # group by elevation rounded
    epd_df['elev_m'] = m.dis.top.array[epd_df.i.astype(int), epd_df.j.astype(int)]
    epd_df['elev_m_grp'] = ((epd_df['elev_m'] *0.5).round(-2))/0.5 # every 200 meters
    # convert the grid referenced x,y to a coordinate system referenced grid
    rot *= (np.pi/180) # convert degrees to radians
    lx = epd_df['x'+loc_id]
    ly = epd_df['y'+loc_id]
    x = lx*np.cos(rot) - ly*np.sin(rot)
    y = ly*np.cos(rot) + lx*np.sin(rot)
    epd_gdf = gpd.GeoDataFrame(epd_df, geometry = gpd.points_from_xy(x+xll, y+yll), crs=crs)
    # summarize by modflow node
    epd_df_mean = epd_df.groupby('node'+loc_id).mean(numeric_only=True).reset_index()
    lx = epd_df_mean['x'+loc_id]
    ly = epd_df_mean['y'+loc_id]
    x = lx*np.cos(rot) - ly*np.sin(rot)
    y = ly*np.cos(rot) + lx*np.sin(rot)
    epd_mean_gdf = gpd.GeoDataFrame(epd_df_mean, geometry = gpd.points_from_xy(x+xll, y+yll), crs=crs)

    return(epd_gdf, epd_mean_gdf)
#     return(epd_gdf)



In [ ]:
def src_area_bnds(epd, aoi):
    if tracking == 'forward':
        loc_id = '0' # with forward tracking initial location is source
    elif tracking =='backward':
        loc_id = '' # with backward tracking final location is source
    from shapely.geometry import box
    outx = xll+epd['x'+loc_id]
    outy = yll+epd['y'+loc_id]
    src_box = box(outx.min(), outy.min(), outx.max() ,  outy.max())
    src_area = aoi[['Name','Hydrologic','geometry']].copy()
    src_area.geometry = [src_box]
    src_area.crs='epsg:3310'
    return(src_area)

In [ ]:
scenario = '' # baseline, levee removal occurred in 2014
# create identifier for scenario if levee removal didn't occur
scenario = 'no_reconnection'

In [ ]:
loadpth = 'C:/WRDAPP/GWFlowModel/Cosumnes/Stream_seepage/'
model_nam = 'oneto_denier_upscale4x_2014_2020'

model_ws = join(loadpth,model_nam)
if scenario != '':
    model_ws += '_' + scenario
    
    
m = flopy.modflow.Modflow.load('MF.nam', model_ws=model_ws, 
                                exe_name='mf-owhm', version='mfnwt') #, load_only = load_only)

nlay = m.dis.nlay
nrow = m.dis.nrow
ncol = m.dis.ncol
xll,yll = m.modelgrid.xoffset,m.modelgrid.yoffset

delr = m.dis.delr.array[0]
delc = m.dis.delc.array[0]

nper = m.dis.nper # number of months
perlen = m.dis.perlen.array # days per period

strt_date = np.datetime64(m.start_datetime)
end_date = strt_date + np.timedelta64(int(perlen.sum()-1),'D')

In [ ]:
hdobj = flopy.utils.HeadFile(model_ws+'/MF.hds')
spd_stp = hdobj.get_kstpkper()
cbb = flopy.utils.CellBudgetFile(model_ws+'/MF.cbc')

times = hdobj.get_times()
# subtract 1 to have start of each period
dt_ref = pd.DataFrame(np.asarray(times), columns=['time']) 
dt_ref['timedelta'] = pd.to_timedelta(dt_ref.time, 'D').round('H')
dt_ref['dt'] = strt_date+dt_ref.timedelta
dt_ref['kstpkper'] = spd_stp
dt_ref[['kstp','kper']] = dt_ref.kstpkper.tolist()

In [ ]:
# run shorter MODPATH simulation to reduce run time
n_days = (dt_ref.dt.max()-dt_ref.dt.min()).days # full time period
tracking = 'forward'
# tracking = 'backward'

mp_ws = join(model_ws,'mp',tracking, 'mp_'+str(n_days)+'days')
modelname = 'Modpath7'
print(basename(mp_ws))

In [ ]:
model_grp = 'inset_oneto_denier'
grid_dir = join(gwfm_dir, 'DIS_data/streambed_seepage/grid')
grid_fn = join(grid_dir, model_grp,'rm_only_grid.shp')
grid_p = gpd.read_file(grid_fn)
grid_p.crs = 'epsg:32610'

In [ ]:
sfrdf = pd.DataFrame(m.sfr.reach_data)
# get geodataframe for sfr
grid_sfr = grid_p.set_index(['row','column']).loc[list(zip(sfrdf.i+1, sfrdf.j+1))]
grid_sfr = pd.concat((grid_sfr.reset_index(), sfrdf), axis=1)
# gdf for lak
lak_row, lak_col = np.where(m.lak.lakarr.array[0,0]>0)
grid_lak = grid_p.set_index(['row','column']).loc[list(zip(lak_row+1, lak_col+1))].reset_index()

In [ ]:
# get nodes for Big Springs at the area of interest scale

aoi_arr = sfrdf[['k','i','j']]
aoi_arr = aoi_arr-1 # convert to 0 based
aoi_ids = list(zip(aoi_arr.k, aoi_arr.i, aoi_arr.j))

nodes_aoi = get_nodes(aoi_ids) # get nodes
print(len(nodes_aoi))

In [ ]:
# the issue that occurred in modpath was that the modpath particle id got so long that in the output file 
# modpath couldn't put a space between it and the particle release number and so it became a bunch of ***
# need to shorten pid or

In [ ]:
# some of the last rows have **** appended into the name of column 1
# error checking
# pd.read_csv(join(mp_ws, modelname + ".mpend"), skiprows=2195, delimiter=r'\s+',engine='python',header=None)

In [ ]:
# p = flopy.utils.PathlineFile(mp_ws + modelname+'.mppth')
e = flopy.utils.EndpointFile(join(mp_ws, modelname + ".mpend"))

# big springs iso
# p_bs = p.get_destination_pathline_data(nodes_bs, to_recarray=True)
# p_bs = p.get_destination_pathline_data(nodes_bs_sfr, to_recarray=True)

# if tracking == 'forward':
#     epd_src = e.get_alldata() # returns the same result because backward tracking is a small initial area
# if tracking == 'backward':
#     epd_src = e.get_destination_endpoint_data(dest_cells=nodes_aoi)
# # epd_src = e.get_destination_endpoint_data(dest_cells=nodes_aoi, source=True)
    
epd_src = e.get_alldata() # returns the same result because backward tracking is a small initial area


In [ ]:
# print((pd.DataFrame(epd_src).time - pd.DataFrame(epd_src).time0).unique())
# print(np.unique(epd_src.status))

In [ ]:
epd_gdf, epd_mean_gdf = epd_2_df(epd_src, rot=52.9, crs=grid_p.crs)
# epd_gdf = epd_2_df(epd_src, rot=52.9, crs=grid_p.crs)

# bs_src_area = src_area_bnds(epd_src, bs_aoi)

In [ ]:
epd_gdf.status_name.unique()

In [ ]:
# particles instantly terminated
# even setting the start layer to 4 all of the particles instantly terminate, may be due to leakage face?
# layer 2 same issue so there is a problem with model set up
# before changing from stop_at to pass_through
# epd_gdf[['k0','xloc0','yloc0','k','xloc','yloc']]
# epd_gdf.status.unique() 

In [ ]:
# epd_gdf[epd_gdf.age_days>0] # 3 stress periods had non zero times, groups 0-2 for times 2.83 to 4.83

In [ ]:
# most are weakly terminated
fig,ax = plt.subplots(figsize=(4,4))
epd_gdf.plot('age_days', ax=ax, legend=True, legend_kwds = {'shrink':0.6,'orientation':'vertical'})
grid_sfr.plot(ax=ax)
grid_lak.plot(ax=ax,color='none',linewidth=0.3)
# plt_cln(ax=ax)

To do:
Look at travel distance in vertical and horizontal.

In [ ]:
epd_gdf['date'] = strt_date + epd_gdf.time0.astype('timedelta64[D]')

In [ ]:
# t = flopy.utils.TimeseriesFile(mp_ws+ modelname + ".timeseries")

# # file is 1.3 Gb
# # ts = t.get_data() # loads nothing
# ts = t.get_destination_timeseries_data(dest_cells=nodes_bs_aoi)
# # create dataframe
# ts_df = pd.DataFrame().from_records(ts)

# Cumulative age plots

In [ ]:
import matplotlib.cm as cm

In [ ]:
# because there are so many days this code takes a long time


# old code from shasta that needs to be updated
#normalize item number values to colormap
norm = mpl.colors.Normalize(vmin=0, vmax=len(epd_flow.date.unique()))

# plot a cumulative age plot for the cumulative three cells
# for each time step
fig,ax = plt.subplots(figsize=(6,6))
# plt.cmap = 'gray'

epd_flow = epd_gdf.copy()
for n, t in enumerate(epd_flow.date.unique()):
    t_epd = pd.DataFrame(epd_flow[epd_flow.date==t].copy())
#     t_epd['weight'] = t_epd.flow/t_epd.flow.sum()
    t_epd = t_epd.sort_values('age_years')
    # temporary weight to decide if need
    t_epd['weight_cum'] = np.linspace(0,1, len(t_epd))
#     t_epd['weight_cum'] = t_epd.weight.cumsum()
    t_epd.plot(x='age_years',y='weight_cum', ax=ax,  label=str(t), color=cm.gray(norm(n)))


In [ ]:
# t_epd

In [ ]:

# old code from shasta that needs to be updated
#normalize item number values to colormap
norm = mpl.colors.Normalize(vmin=0, vmax=len(epd_flow.date.unique()))

# plot a cumulative age plot for the cumulative three cells
# for each time step
fig,ax = plt.subplots(figsize=(6,6))
plt.cmap = 'gray'

for n, t in enumerate(epd_flow.date.unique()):
    t_epd = pd.DataFrame(epd_flow[epd_flow.date==t].copy())
    t_epd['weight'] = t_epd.flow/t_epd.flow.sum()
    t_epd = t_epd.sort_values('age_years')
    t_epd['weight_cum'] = t_epd.weight.cumsum()

    t_epd.plot(x='age_years',y='weight_cum', ax=ax,  label=str(t), color=cm.gray(norm(n)))
ax.set_xlabel('Age (years)')
ax.set_xscale('log') 
ax.set_ylabel('Flow Weighted Fraction of Particles')
ax.set_title('Cumulative Age Distribution\nby Stress Period')


custom_lines = [Line2D([0], [0], color=cm.gray(norm(0)), lw=4),
                Line2D([0], [0], color=cm.gray(norm(n)), lw=4)]
ax.legend(custom_lines, [epd_dates[0], epd_dates[n]], facecolor='lightgray', framealpha=1)

# plt.savefig(fig_dir+'/'+tracking+'/big springs cumulative age average' +  mp_info['strt_date'].values[0]+ '.png',
#             dpi=300, bbox_inches='tight')